# Web Scraping no site da UEA

O objetivo deste scraping é coletar os dados referentes às grades curriculares dos cursos da UEA e reorganizá-los em arquivos _csv_.

Deixo como referência principal o tutorial de scraping do [RodrigoCMoraes](1), que além de ser simples, também referencia outros bons tutoriais. Recomendo!

[1]:https://github.com/RodrigoCMoraes/web_scraping/blob/master/download_editais_fapeam/Download%20de%20Editais%20Vigentes%20FAPEAM.ipynb

<h1>Tabela de Conteúdos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Primeira-página-de-login" data-toc-modified-id="Primeira-página-de-login-1">Primeira página de login</a></span></li><li><span><a href="#Segunda-página-de-login-(???)" data-toc-modified-id="Segunda-página-de-login-(???)-2">Segunda página de login (???)</a></span></li><li><span><a href="#Página-com-informação-sobre-os-cursos" data-toc-modified-id="Página-com-informação-sobre-os-cursos-3">Página com informação sobre os cursos</a></span></li><li><span><a href="#Grade-curricular" data-toc-modified-id="Grade-curricular-4">Grade curricular</a></span></li><li><span><a href="#Tratamento-dos-dados" data-toc-modified-id="Tratamento-dos-dados-5">Tratamento dos dados</a></span></li></ul></div>

In [1]:
import re, sys, requests, json
from bs4 import BeautifulSoup, Tag
from ast import literal_eval
import pandas as pd
sys.setrecursionlimit(10000)

In [2]:
# Funções auxiliares

def fix(s):
    if s is None:
        s = ''
    return re.sub(' \w\w? ', ',', s)

def is_grade_curricular(tag):
    return tag.name == 'a' and tag.next_element == 'Grade Curricular'

# mostra o histórico dos redirecionamentos
def track(last_page):
    for page in last_page.history+[last_page]:
        print(f'{page}: {page.url} (text_len={len(page.text)})')

In [3]:
# importação de pacotes base
import cipher        # local
import getpass, json # padrão do python

encrypt_base_string = 'encryptbasestring'

# aquisição de informações
matricula = input('matricula: ')
username = input('username: ')
password = getpass.getpass('password: ')

# encriptação da senha
password_encrypted = cipher.encrypt(encrypt_base_string, password)

# só por "segurança"
del password

credentials = {
    'login': {
        'username': username,
        'password': password_encrypted
    }, 
    'middle_logon': {
        'txtnumero_matricula': matricula, 
        'txtsenha_tac': 'facada0123456789fedcba0123456789',
        'txtsenha': 'facada0123456789fedcba0123456789'
    }
}

with open('credentials.json', 'w') as outfile:
    json.dump(credentials, outfile)

matricula: 1315208157
username: rcm.eng@uea.edu.br
password: ········


In [4]:
session = requests.Session()
config = json.load(open('credentials.json'))
print(json.dumps(config, indent=2))

{
  "login": {
    "username": "rcm.eng@uea.edu.br",
    "password": "36be183be3af4e16-0154cb1f8b8d936913f87ffe-6b86b7a8ec050f5f5b66d953c7b3facdfd3b0607bb05d1ed6f155a12f1e7828303a0f8fd3541"
  },
  "middle_logon": {
    "txtnumero_matricula": "1315208157",
    "txtsenha_tac": "facada0123456789fedcba0123456789",
    "txtsenha": "facada0123456789fedcba0123456789"
  }
}


### Primeira página de login

Para saber qual as chaves do payload, basta inspecionar o elemento com o mouse em cima da caixa de login.

In [5]:
password = cipher.decrypt(encrypt_base_string, config['login']['password'])

payload_login = dict(
    username=config['login']['username'],
    password=password)
del password

url_login = 'http://www1.uea.edu.br/modulo/login/lyceum2.php?'
result = session.post(url_login, data=payload_login, headers=dict(referer=url_login))
track(result)

<Response [200]>: http://www1.uea.edu.br/modulo/login/lyceum2.php (text_len=4304)


### Segunda página de login (???)

Aqui as coisas ficam um pouco estranhas (ou eu realmente não tenho conhecimento sobre essa "técnica").

Basicamente, se você analisar a aba _networking_ do _inspecionar elemento_ enquanto faz o login no aluno online, o site rapidamente o redireciona para outra página de login, antes de realizar o login de fato. Essa seguna página de login é preenchida automaticamente, mas o que me preocupa é que para logar no aluno online, **você não precisa logar na primeira página, apenas na segunda**.

Isso é, observando o payload dessa requisição POST, o necessário para alguém entrar na sua conta é saber sua matrícula, seu *txtsenha_tac* e sua *txtsenha*. Não sei o que são esses dois últimos parâmetros, mas eu espero que seja uma chave gerada utilizando nosso email institucional + senha.

Vale notar também que essas possíveis chaves são constantes, logo, não mudam por sessão. Bom, fica o mistério do porquê é implementado desse jeito (pelo menos para mim).

Segue uma pequena demonstração de como pegar o valor desses parâmetros para _seu_ login:
![](media/middle_logon.gif)

Será que podemos automatizar? :)

In [6]:
payload_middle_logon = dict(
    txtnumero_matricula=config['middle_logon']['txtnumero_matricula'],
    txtsenha_tac       =config['middle_logon']['txtsenha_tac'],
    txtsenha           =config['middle_logon']['txtsenha'])
url_middle_logon = 'http://www1.uea.edu.br/lyceump/aonline/middle_logon.asp'
result = session.post(url_middle_logon, data=payload_middle_logon, headers=dict(refer=url_middle_logon))
track(result)

<Response [302]>: http://www1.uea.edu.br/lyceump/aonline/middle_logon.asp (text_len=132)
<Response [302]>: http://www1.uea.edu.br/lyceump/aonline/default.asp (text_len=138)
<Response [302]>: http://www1.uea.edu.br/lyceump/aonline/questionarios.asp (text_len=131)
<Response [200]>: http://www1.uea.edu.br/lyceump/aonline/avisos.asp (text_len=48121)


### Página com informação sobre os cursos

![cursos](media/cursos.png)

In [7]:
url_cursos = 'http://www1.uea.edu.br/lyceump/aonline/cursos.asp'
result = session.get(url_cursos)
content = BeautifulSoup(result.content, 'lxml')
track(result)

<Response [200]>: http://www1.uea.edu.br/lyceump/aonline/cursos.asp (text_len=1265008)


### Grade curricular

Inicialmente seleciono apenas uma grade de um curso específico, mas é possível generalizar para todos as grades sem muita dificuldade.

![grade](media/grade.png)

In [8]:
links = content.find_all(is_grade_curricular)
link = links[1874] # grade atual de SI

keys = ['cursonome', 'curso', 'turno', 'curriculo', 'alt']
params_raw = re.search('\((.*)\);$', link['href']).groups(1)[0]
params = list(literal_eval(params_raw))
payload = {key: val for key, val in zip(keys, params)}
print(payload)
url_grade = 'http://www1.uea.edu.br/lyceump/aonline/grade.asp'
result = session.post(url_grade, data=payload, headers=dict(refer=url_grade))
track(result)

{'cursonome': 'Sistemas de Informação', 'curso': 'EST31MSII', 'turno': 'INT', 'curriculo': 'BSII_2018', 'alt': 'Mostrar grade do curriculo'}
<Response [200]>: http://www1.uea.edu.br/lyceump/aonline/grade.asp (text_len=61317)


### Tratamento dos dados

In [9]:
# Não usar lxml como parser: https://stackoverflow.com/a/21655159
content = BeautifulSoup(result.content, 'html5lib')

In [10]:
# Seleciona a primeira linha "amarela" na tabela
# Depois, itera sobre as linhas da tabela e trata as informações relevantes
first_row = content.find(class_='tr01t')
relevant_rows = first_row.find_next_siblings(class_=['font02', 'tr01', 'tr01a'])
periodo = 1
disciplinas = []
for row in relevant_rows:
    if (row['class'] == ['font02']):
        periodo += 1
    else:
        cod_nome, prereqs, equivs = (dado.string for dado in row.children)
        codigo = cod_nome.split()[0]
        nome = " ".join(cod_nome.split()[2:])
        prereqs = fix(prereqs)
        equivs = fix(equivs)
        disciplinas.append([codigo, nome, prereqs, equivs, periodo])

columns = ['cod', 'nome', 'prereq', 'equiv', 'periodo']
grade = pd.DataFrame(disciplinas, columns=columns).set_index('cod')
grade # Precisa de redução transitiva.

,nome,prereq,equiv,periodo
cod,,,,
ESTCMP001,Introdução a Computação,,ESTSIV004,1
ESTCMP009,Introdução a Programação de Computadores,,"ESTSIV006,ESTTPD108,ESTECP001",1
ESTCMP028,Matemática Básica,,ESTSIV007,1
ESTCMP050,"Computação, Ética, Homem e Sociedade",,"ESTSIV001,ESTBAS022",1
ESTBAS004,Introdução às Ciências do Ambiente,,ESTSIV002,1
ESTBAS011,Introdução à Administração,,ESTSIV003,1
ESTBAS017,Introdução à Economia,,ESTSIV005,1
ESTCMP002,Organização e Arquitetura de Computadores,ESTCMP001,"ESTSIV011,ESTLIN325,ESTECP008,EST0244,ESTTPD244",2
ESTCMP010,Programação de Computadores e Algoritmos,ESTCMP009,"ESTSIV008,ESTTPD245,ESTTPD301,ESTECP002",2


In [11]:
grade.to_csv('data.csv')